In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Biosppy for signal processing
from biosppy.signals import ecg

# Constants
SAMPLING_RATE = 500  # Hz
ecg_folder = "../../../../Datasets/12-lead electrocardiogram database/ECGData"
diagnostics_file = "../../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"

# Label mapping
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}


# Feature extraction function
def extract_comprehensive_features(ecg_signal):
    """
    Extract comprehensive features from entire ECG signal
    """
    try:
        ecg_processed = ecg.ecg(signal=ecg_signal, sampling_rate=SAMPLING_RATE, show=False)
        rpeaks = ecg_processed['rpeaks']
        heart_rate = ecg_processed['heart_rate']

        features = [
            np.mean(ecg_signal),
            np.std(ecg_signal),
            np.median(ecg_signal),
            np.min(ecg_signal),
            np.max(ecg_signal),
            len(rpeaks),
            np.mean(heart_rate) if len(heart_rate) > 0 else 0,
            np.std(heart_rate) if len(heart_rate) > 0 else 0,
            np.mean(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.std(np.diff(rpeaks)) if len(rpeaks) > 1 else 0,
            np.percentile(ecg_signal, 25),
            np.percentile(ecg_signal, 75),
        ]

        return features

    except Exception as e:
        print(f"Error processing signal: {e}")
        return [0] * 12


# Dataset preparation function
def prepare_dataset(ecg_folder, diagnostics_df):
    signals, signal_features, signal_labels = [], [], []

    for _, row in diagnostics_df.iterrows():
        file_name = row['FileName']
        rhythm_label = row['Rhythm']

        if pd.isnull(rhythm_label) or rhythm_label not in rhythm_mapping.values():
            continue

        ecg_file = os.path.join(ecg_folder, f"{file_name}.csv")
        if not os.path.exists(ecg_file):
            continue

        ecg_data = pd.read_csv(ecg_file, header=0).iloc[:, 1].values
        ecg_data = ecg_data.astype(float)

        features = extract_comprehensive_features(ecg_data)
        signals.append(ecg_data)
        signal_features.append(features)
        signal_labels.append(rhythm_label)

    return np.array(signals), np.array(signal_features), np.array(signal_labels)


# Load and process data
diagnostics_df = pd.read_excel(diagnostics_file)
diagnostics_df['Rhythm'] = diagnostics_df['Rhythm'].map(rhythm_mapping)

raw_signals, signal_features, signal_labels = prepare_dataset(ecg_folder, diagnostics_df)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(signal_labels)
onehot_labels = to_categorical(encoded_labels)


# Create MLP model
def create_mlp_model(input_shape, num_classes):
    mlp = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return mlp


# Stratified K-Fold Cross-Validation with optimizer tuning
optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
results = {}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for optimizer in optimizers:
    print(f"\nTesting optimizer: {optimizer}")
    cv_scores = []

    for train_index, test_index in skf.split(signal_features, encoded_labels):
        X_train, X_test = signal_features[train_index], signal_features[test_index]
        y_train, y_test = onehot_labels[train_index], onehot_labels[test_index]

        mlp = create_mlp_model(input_shape=X_train.shape[1], num_classes=onehot_labels.shape[1])
        mlp.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

        history = mlp.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, verbose=1)
        _, accuracy = mlp.evaluate(X_test, y_test, verbose=0)
        cv_scores.append(accuracy)

        y_pred = mlp.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_test_classes = np.argmax(y_test, axis=1)

        print("\nClassification Report:")
        print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_, digits=5))

    results[optimizer] = np.mean(cv_scores)
    print(f"Optimizer: {optimizer} - Mean CV Accuracy: {np.mean(cv_scores)}")

print("\nFinal Results:")
for opt, acc in results.items():
    print(f"{opt}: {acc}")

2024-11-29 07:03:51.723021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 07:03:51.733894: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 07:03:51.737169: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 07:03:51.746174: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-29 07:03:52.260645: W tensorflow/compiler/tf2


Testing optimizer: adam
Epoch 1/100


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1732842539.641747  298570 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732842539.674538  298570 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1732842539.

 91/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3470 - loss: 35.9564

I0000 00:00:1732842541.734916  301954 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

Epoch 1/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.2630 - loss: 46.9700 - val_accuracy: 0.2788 - val_loss: 15.8762
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - accuracy: 0.3356 - loss: 23.4183 - val_accuracy: 0.3011 - val_loss: 11.3996
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - accuracy: 0.3390 - loss: 17.7075 - val_accuracy: 0.3093 - val_loss: 9.3533
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step - accuracy: 0.3551 - loss: 15.1129 - val_accuracy: 0.3192 - val_loss: 7.7298
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step - accuracy: 0.3577 - loss: 13.1902 - val_accuracy: 0.3304 - val_loss: 6.8391
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.3679 - loss: 11.7288 - val_accuracy: 0.3316 - val_loss: 6.1971
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - accuracy: 0.3726 - loss: 10.2563 - val_accuracy: 0.3327 - val_loss: 5.6293
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step - accuracy: 0.3688 -

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2863 - loss: 59.1643 - val_accuracy: 0.2629 - val_loss: 17.1869
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3219 - loss: 29.6862 - val_accuracy: 0.2905 - val_loss: 10.4912
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.3220 - loss: 23.7497 - val_accuracy: 0.2969 - val_loss: 9.0392
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.3403 - loss: 20.2337 - val_accuracy: 0.3140 - val_loss: 7.2605
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - accuracy: 0.3457 - loss: 17.1353 - val_accuracy: 0.3058 - val_loss: 7.0397
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step - accuracy: 0.3472 - loss: 15.3788 - val_accuracy: 0.3163 - val_loss: 5.9281
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - accuracy: 0.3585 - loss: 14.1605 - val_accuracy: 0.3146 - val_loss: 5.1767
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.3736 - loss: 13.3258

/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.2623 - loss: 62.0790 - val_accuracy: 0.4231 - val_loss: 10.5440
Epoch 2/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step - accuracy: 0.3305 - loss: 28.9451 - val_accuracy: 0.4759 - val_loss: 6.2328
Epoch 3/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - accuracy: 0.3658 - loss: 22.6875 - val_accuracy: 0.5164 - val_loss: 4.5349
Epoch 4/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - accuracy: 0.3781 - loss: 19.1208 - val_accuracy: 0.5282 - val_loss: 4.0616
Epoch 5/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 813us/step - accuracy: 0.3847 - loss: 16.7773 - val_accuracy: 0.5335 - val_loss: 4.2170
Epoch 6/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - accuracy: 0.3711 - loss: 15.3756 - val_accuracy: 0.5540 - val_loss: 4.1457
Epoch 7/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 859us/step - accuracy: 0.3859 - loss: 14.0007 - val_accuracy: 0.5563 - val_loss: 3.8325
Epoch 8/100
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step - accuracy: 0.3822 - loss: 13.373